In [2]:
from itertools import permutations, product

import tqdm

import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
from os import path
import numpy as np
import matplotlib.pyplot as plt

from warnings import simplefilter
from collections import OrderedDict

import pickle

if (os.path.abspath('').split('/')[-1] == 'project'):
    %cd utils
elif (os.path.abspath('').split('/')[-1] == 'train_and_vis'):
    %cd ../utils

import query_utils
import model_utils
import validation_utils
import data_utils

print(os.path.abspath(''))

if (os.path.abspath('').split('/')[-1] == 'utils'):
    %cd ..


simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)

/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project


In [3]:
dataset = data_utils.retrieveAllDatasets()

working directory: /Users/benjaminli/Documents/coding/scires/project
kmer: 3
zhang reg 9329
zhang norm 9329
nardus reg 1848
nardus norm 1848
merge reg 10676
merge norm 10676
kmer: 4
zhang reg 9329
zhang norm 9329
nardus reg 1848
nardus norm 1848
merge reg 10676
merge norm 10676
kmer: 5
zhang reg 9329
zhang norm 9329
nardus reg 1848
nardus norm 1848
merge reg 10676
merge norm 10676
kmer: 6
zhang reg 9329
zhang norm 9329
nardus reg 1848
nardus norm 1848
merge reg 10676
merge norm 10676


In [15]:
merged = dataset['merged']['regular-4']
mergedX = merged['X_test']
mergedY = merged['y_test']

model = pickle.load(open('models/curr_models/final_merged_stackingcv.pkl', 'rb'))

x=0
di_zoo = {"1000": 0, "3000": 0, "5000": 0, "10000": 0, ">10000": 0}
di_nonzoo = {"1000": 0, "3000": 0, "5000": 0, "10000": 0, ">10000": 0}

di_overall = {"1000": 0, "3000": 0, "5000": 0, "10000": 0, ">10000": 0}

for r in range(len(mergedX)):
    if (mergedX.iloc[r].sum() <= 1000):        
        if mergedY[r] == 1: di_zoo["1000"] += 1
        else: di_nonzoo["1000"] += 1
        di_overall["1000"] += 1

    elif (mergedX.iloc[r].sum() <= 3000):
        if mergedY[r] == 1: di_zoo["3000"] += 1
        else: di_nonzoo["3000"] += 1

        di_overall["3000"] += 1

    elif (mergedX.iloc[r].sum() <= 5000):
        if mergedY[r] == 1: di_zoo["5000"] += 1
        else: di_nonzoo["5000"] += 1

        di_overall["5000"] += 1

    elif (mergedX.iloc[r].sum() <= 10000):
        if mergedY[r] == 1: di_zoo["10000"] += 1
        else: di_nonzoo["10000"] += 1

        di_overall["10000"] += 1

    else:
        if mergedY[r] == 1: di_zoo[">10000"] += 1
        else: di_nonzoo[">10000"] += 1

        di_overall[">10000"] += 1
print(di_zoo)
print(di_nonzoo)


mergedXNormed = mergedX.apply(lambda x: (x-x.min())/(x.max()-x.min()), axis=1)

pred_di_zoo = {"1000": 0, "3000": 0, "5000": 0, "10000": 0, ">10000": 0}
pred_di_nonzoo = {"1000": 0, "3000": 0, "5000": 0, "10000": 0, ">10000": 0}
pred_di_overall = {"1000": 0, "3000": 0, "5000": 0, "10000": 0, ">10000": 0}
preds = []
for r in tqdm.tqdm(range(len(mergedXNormed))):
    data = pd.DataFrame([mergedXNormed.iloc[r].to_list()], columns=mergedXNormed.iloc[r].index)
    # print(data)
    pred_proba = model.predict_proba(data)[:,1]
    pred = 1 if pred_proba >= 0.5 else 0

    # print(pred_proba)
    # print(pred)
    preds.append(pred_proba)
    if (mergedX.iloc[r].sum() <= 1000): # perform check on the regular dataset
        if (mergedY[r] == pred and pred == 1):
            pred_di_zoo["1000"] += 1
        elif (mergedY[r] == pred and pred == 0):
            pred_di_nonzoo["1000"] += 1
        if (mergedY[r] == pred):
            pred_di_overall["1000"] += 1

    elif (mergedX.iloc[r].sum() <= 3000):
        if (mergedY[r] == pred and pred == 1):
            pred_di_zoo["3000"] += 1
        elif (mergedY[r] == pred and pred == 0):
            pred_di_nonzoo["3000"] += 1
        if (mergedY[r] == pred):
            pred_di_overall["3000"] += 1
        
    elif (mergedX.iloc[r].sum() <= 5000):
        if (mergedY[r] == pred and pred == 1):
            pred_di_zoo["5000"] += 1
        elif (mergedY[r] == pred and pred == 0):
            pred_di_nonzoo["5000"] += 1
        if (mergedY[r] == pred):
            pred_di_overall["5000"] += 1
        
    elif (mergedX.iloc[r].sum() <= 10000):
        if (mergedY[r] == pred and pred == 1):
            pred_di_zoo["10000"] += 1
        elif (mergedY[r] == pred and pred == 0):
            pred_di_nonzoo["10000"] += 1
        if (mergedY[r] == pred):
            pred_di_overall["10000"] += 1
        
    else:
        if (mergedY[r] == pred and pred==1):
            pred_di_zoo[">10000"] += 1
        elif (mergedY[r] == pred and pred == 0):
            pred_di_nonzoo[">10000"] += 1
        if (mergedY[r] == pred):
            pred_di_overall[">10000"] += 1
        

{'1000': 8, '3000': 45, '5000': 28, '10000': 182, '>10000': 66}
{'1000': 11, '3000': 290, '5000': 189, '10000': 460, '>10000': 857}


100%|██████████| 2136/2136 [00:38<00:00, 55.59it/s]


In [5]:
print("zoonotic")
for key in di_zoo:
    print(key, di_zoo[key], pred_di_zoo[key]/di_zoo[key])

print("nonzoonotic")
for key in di_nonzoo:
    print(key,di_nonzoo[key], pred_di_nonzoo[key]/di_nonzoo[key])

print("overall")
for key in di_overall:
    print(key,di_overall[key], pred_di_overall[key]/di_overall[key], pred_di_overall[key])

print("total zoonotic predictions: " + str(sum(pred_di_zoo.values())))

zoonotic
1000 8 0.75
3000 45 0.8222222222222222
5000 28 0.8214285714285714
10000 182 0.9615384615384616
>10000 66 0.9090909090909091
nonzoonotic
1000 11 1.0
3000 290 0.993103448275862
5000 189 0.9947089947089947
10000 460 0.9956521739130435
>10000 857 0.9953325554259043
overall
1000 19 0.8947368421052632 17
3000 335 0.9701492537313433 325
5000 217 0.9723502304147466 211
10000 642 0.985981308411215 633
>10000 923 0.9891657638136512 913
total zoonotic predictions: 301


In [20]:
# sort by probability
pred_proba = sorted(preds, reverse=True)
print(pred_proba)

[array([0.97084031]), array([0.97060904]), array([0.97060079]), array([0.97059022]), array([0.97057468]), array([0.97056539]), array([0.97056274]), array([0.97055195]), array([0.97055139]), array([0.97053705]), array([0.97049976]), array([0.97046929]), array([0.97046451]), array([0.97044221]), array([0.97043462]), array([0.97041885]), array([0.97041605]), array([0.97038129]), array([0.97037154]), array([0.97035035]), array([0.97031994]), array([0.97030827]), array([0.97030678]), array([0.97030673]), array([0.97030283]), array([0.97027042]), array([0.97025059]), array([0.97024257]), array([0.97023696]), array([0.97023291]), array([0.9702157]), array([0.97021064]), array([0.97016736]), array([0.97013903]), array([0.97013317]), array([0.9701216]), array([0.97010684]), array([0.97009995]), array([0.97009404]), array([0.97009291]), array([0.97008089]), array([0.97007]), array([0.9700673]), array([0.9700486]), array([0.97002138]), array([0.97002122]), array([0.96999285]), array([0.96994485])

In [13]:
# print(19/(19+335+217+642+923))
df = pd.DataFrame()
# fill df with the bp lengths as the index and the number of zoonotic and nonzoonotic as the columns
df['zoonotic'] = pd.Series(di_zoo)
df['pred_zoonotic'] = pd.Series(pred_di_zoo)
df['overall'] = pd.Series(di_overall)
df['pred_overall'] = pd.Series(pred_di_overall)
df['pred_overall%'] = df['pred_overall']/df['overall']

df.head()

,zoonotic,pred_zoonotic,overall,pred_overall,pred_overall%
1000,8,6,19,17,0.894737
3000,45,37,335,325,0.970149
5000,28,23,217,211,0.972350
10000,182,175,642,633,0.985981
>10000,66,60,923,913,0.989166
